In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

### 0. Загрузка данных

In [3]:
!pip install mne
!wget https://raw.githubusercontent.com/adasegroup/NEUROML2020/seminar1/seminar1/train.csv
!wget https://raw.githubusercontent.com/adasegroup/NEUROML2020/seminar1/seminar1/test.csv

--2025-12-28 21:31:19--  https://raw.githubusercontent.com/adasegroup/NEUROML2020/seminar1/seminar1/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5381138 (5.1M) [text/plain]
Saving to: ‘train.csv.1’

train.csv.1         100%[===================>]   5.13M  --.-KB/s    in 0.08s   

2025-12-28 21:31:19 (61.7 MB/s) - ‘train.csv.1’ saved [5381138/5381138]

--2025-12-28 21:31:19--  https://raw.githubusercontent.com/adasegroup/NEUROML2020/seminar1/seminar1/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3567877 (3.4M

In [4]:
!pip install lightautoml

In [6]:
import pandas as pd
import numpy as np
import mne

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

from src.dataset import get_target, calc_features

delimiter = "\n" + "-"*10

In [8]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

,time,condition,epoch,C3,Cz,C4,Fp1,Fp2,F7,F3,...,F8,T7,T8,P7,P3,Pz,P4,P8,O1,O2
0,0,1,0,-5.885714,-2.533107,9.866895,7.962973,5.694433,23.638605,27.899784,...,23.588723,12.178548,0.685809,-4.887397,-10.646985,-14.735646,-8.729323,-0.562578,-17.055458,-3.616732
1,1,1,0,-7.999715,-16.916729,-11.924855,17.955477,8.526994,56.635981,28.508435,...,1.045533,14.656061,-4.119778,-4.632381,-17.980657,-23.456960,-12.960684,-9.639784,-20.233549,-1.229811
2,2,1,0,-6.727283,-15.979567,-11.114195,17.183478,4.497028,43.914130,10.079754,...,-14.741630,14.793562,-6.624813,-3.402757,-10.269473,-18.736144,-3.579046,-0.045658,-14.089755,2.453398
3,3,1,0,6.819390,-0.204905,10.090124,20.265222,7.843006,36.250611,13.291199,...,-7.135541,21.723418,-2.276825,2.066859,4.325365,-2.803322,8.835114,13.878945,-5.772410,7.657873
4,4,1,0,13.129486,-5.817193,5.040633,19.462210,9.634234,42.311729,20.641012,...,0.015602,19.703190,3.739076,2.714350,3.251047,-3.631448,3.212956,4.635574,-5.045448,5.086024


In [9]:
non_eeg = ['time', 'condition', 'epoch', 'target']
channels = [ch for ch in df_train.columns if ch not in non_eeg]

In [13]:
X = get_target(df_train)
X = X.merge(calc_features(df_train, channels), on='epoch')
y = X['condition'].apply(lambda x: 0 if x == 1 else 1)

X = X.drop(['epoch', 'condition'], axis=1)
y = y.astype(int)
y.name = 'target'
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

## 3. Моделирование (максимум 3.5 балла)
[0.25] Обоснование стратегии разделения данных (train-test split)
Особое внимание уделить предотвращению утечки данных

[0.25] LAMA бейзлайн:
- Минимум 2 различные конфигурации
- Выбор лучшего решения

[3.0] Собственное решение (если не удалось побить LLama baseline: 3 x 1.0 балл за различные пайплайны/попытки):
- Выбор модели
- Построение пайплайна (препроцессинг, обработка пропусков, генерация признаков, отбор признаков, финальная модель/ансамбль)
- Оптимизация гиперпараметров

In [14]:
# Cтратегии разделения данных:
# В соответствием с выводами из файла EDA (part 1) делаем корректный train-test split по данным, предварительно подготовленным по эпохам
npr = np.random.RandomState(42)
epoch_list = X.index
train_idx, test_idx = train_test_split(epoch_list, test_size=0.33, random_state=42) ### делаем разбиение по эпохам как независимым объектам, чтобы избежать утечки между зависимыми наблюдениями, поскольку эпохи коррелированы внутри себя

X_train = X.iloc[train_idx]
X_test = X.iloc[test_idx]
y_train = y.iloc[train_idx]
y_test = y.iloc[test_idx]


In [15]:
# Подготовим данные для конфигураций
roles = {'target': 'target'}
task = Task('binary')

train_LAMA = X_train.copy()
train_LAMA['target'] = y_train.values

test_LAMA = X_test.copy()
test_LAMA['target'] = y_test.values

#### Конфигурация 1 - Классический LAMA (default)
Стандартная конфигурация - подбор лучших моделей (LGB, CatBoost и др.) с автотюнингом. Используем пресеты "по умолчанию".

In [16]:
automl1 = TabularAutoML(
    task=task,
    timeout=600,
    cpu_limit=2,
)
oof_pred1 = automl1.fit_predict(train_LAMA, roles=roles, verbose=2)
test_pred1 = automl1.predict(test_LAMA)

auc1 = roc_auc_score(y_test, test_pred1.data[:, 0])
f1_1 = f1_score(y_test, test_pred1.data[:, 0] > 0.5)
print(f"[LAMA Default] ROC-AUC: {auc1:.4f} F1: {f1_1:.4f}")

[21:33:26] Stdout logging level is INFO2.


INFO:lightautoml.automl.presets.base:Stdout logging level is INFO2.


[21:33:26] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer


[21:33:26] Task: binary



INFO:lightautoml.automl.presets.base:Task: binary



...

[21:34:17] Automl preset training completed in 51.40 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 51.40 seconds



[21:34:17] Model description:
Final prediction for new objects (level 0) = 
	 0.61633 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.07663 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.30704 * (5 averaged models Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 0.61633 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.07663 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.30704 * (5 averaged models Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost) 



[LAMA Default] ROC-AUC: 0.7217 F1: 0.6557


### Конфигурация 2 — FAST, только lgbm
Используем только быстрый LightGBM.

In [17]:
automl2 = TabularAutoML(
    task=task,
    timeout=300,
    cpu_limit=2,
    general_params={
        'use_algos': [['lgb']]
    }
)
oof_pred2 = automl2.fit_predict(train_LAMA, roles=roles, verbose=1)
test_pred2 = automl2.predict(test_LAMA)

auc2 = roc_auc_score(y_test, test_pred2.data[:, 0])
f1_2 = f1_score(y_test, test_pred2.data[:, 0] > 0.5)
print(f'[LAMA FAST (lgb only)] ROC-AUC: {auc2:.4f}  F1: {f1_2:.4f}')


[21:34:18] Stdout logging level is INFO.


INFO:lightautoml.automl.presets.base:Stdout logging level is INFO.


[21:34:18] Task: binary



INFO:lightautoml.automl.presets.base:Task: binary



[21:34:18] Start automl preset with listed constraints:


...

[21:34:18] Automl preset training completed in 0.48 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 0.48 seconds



[21:34:18] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LightGBM) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LightGBM) 



[LAMA FAST (lgb only)] ROC-AUC: 0.7138  F1: 0.6552


### Конфигурация 3 - расширенный набор моделей и мощный автотюнинг

Включаем сразу несколько ансамблей (XGB, LGBM с тюнингом, CatBoost с тюнингом, MLP нейросеть) + используем продвинутую кросс-валидацию, автотюнинг с разными алгоритмами

In [18]:
automl3 = TabularAutoML(
    task=task,
    timeout=600,
    cpu_limit=2,
    general_params={
        'use_algos': [['xgb', 'lgb_tuned', 'cb_tuned'], ['mlp']],  # Много моделей
    },
    reader_params={
        'n_jobs': 2,
        'cv': 5,
        'random_state': 84,
        'advanced_roles': True,
    },
    tuning_params={'max_tuning_iter': "auto", 'max_tuning_time': 300},
    selection_params={'mode': 1}
)

oof_pred3 = automl3.fit_predict(train_LAMA, roles=roles, verbose=1)
test_pred3 = automl3.predict(test_LAMA)

auc3 = roc_auc_score(y_test, test_pred3.data[:, 0])
f1_3 = f1_score(y_test, test_pred3.data[:, 0] > 0.5)
print(f'[LAMA Advanced] ROC-AUC: {auc3:.4f}  F1: {f1_3:.4f}')


[21:34:18] Stdout logging level is INFO.


INFO:lightautoml.automl.presets.base:Stdout logging level is INFO.


[21:34:18] Task: binary



INFO:lightautoml.automl.presets.base:Task: binary



[21:34:18] Start automl preset with listed constraints:


...

[21:34:53] Automl preset training completed in 34.80 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 34.80 seconds



[21:34:53] Model description:
Models on level 0:
	 5 averaged models Lvl_0_Pipe_0_Mod_0_Tuned_LightGBM
	 5 averaged models Lvl_0_Pipe_0_Mod_1_Tuned_CatBoost
	 5 averaged models Lvl_0_Pipe_0_Mod_2_XGBoost

Final prediction for new objects (level 1) = 
	 1.00000 * (5 averaged models Lvl_1_Pipe_0_Mod_0_TorchNN_mlp_0) 



INFO:lightautoml.automl.presets.base:Model description:
Models on level 0:
	 5 averaged models Lvl_0_Pipe_0_Mod_0_Tuned_LightGBM
	 5 averaged models Lvl_0_Pipe_0_Mod_1_Tuned_CatBoost
	 5 averaged models Lvl_0_Pipe_0_Mod_2_XGBoost

Final prediction for new objects (level 1) = 
	 1.00000 * (5 averaged models Lvl_1_Pipe_0_Mod_0_TorchNN_mlp_0) 



[LAMA Advanced] ROC-AUC: 0.5000  F1: 0.0000


### Конфигурация 4 - с feature selection и ограниченным набором моделей
Только LGBM (c hyperparameter tuning) и линейная регрессия; включён feature selection (оставляем 80% лучших признаков); гиперпараметры подбираются на holdout-валидации.


In [19]:
automl4 = TabularAutoML(
    task=task,
    timeout=600,
    cpu_limit=2,
    general_params={
        'use_algos': [['lgb_tuned', 'linear_l2']]  # Только LGBM + Linear
    },
    selection_params={
        'select_algos': ['gbm', 'linear_l2'],
        'selection_feats_rate': 0.8,
        'max_features_cnt_in_result': None,
    },
    tuning_params={
        'fit_on_holdout': True,
        'max_tuning_iter': 50,
        'max_tuning_time': 200,
    }
)

oof_pred4 = automl4.fit_predict(train_LAMA, roles=roles, verbose=1)
test_pred4 = automl4.predict(test_LAMA)

auc4 = roc_auc_score(y_test, test_pred4.data[:, 0])
f1_4 = f1_score(y_test, test_pred4.data[:, 0] > 0.5)
print(f'[LAMA FS+LGBM/Linear] ROC-AUC: {auc4:.4f}  F1: {f1_4:.4f}')


[21:34:56] Stdout logging level is INFO.


INFO:lightautoml.automl.presets.base:Stdout logging level is INFO.


[21:34:56] Task: binary



INFO:lightautoml.automl.presets.base:Task: binary



[21:34:56] Start automl preset with listed constraints:


...

[21:35:01] Automl preset training completed in 5.54 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 5.54 seconds



[21:35:01] Model description:
Final prediction for new objects (level 0) = 
	 0.66234 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.33766 * (5 averaged models Lvl_0_Pipe_1_Mod_0_Tuned_LightGBM) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 0.66234 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.33766 * (5 averaged models Lvl_0_Pipe_1_Mod_0_Tuned_LightGBM) 



[LAMA FS+LGBM/Linear] ROC-AUC: 0.7217  F1: 0.4082


#### Выбор лучшего

In [20]:
results = {}
results['LAMA_DEFAULT'] = (auc1, f1_1)
results['LAMA_FAST'] = (auc2, f1_2)
results['LAMA_ADVANCED'] = (auc3, f1_3)
results['LAMA_FS'] = (auc4, f1_4)
for k, v in results.items():
    print(f'{k:15}| ROC-AUC: {v[0]:.4f} | F1: {v[1]:.4f}')

best_lama_name = max(results, key=lambda k: results[k][0])
print(f'--> Лучший бэйзлайн: {best_lama_name}, ROC-AUC = {results[best_lama_name][0]:.4f}')

LAMA_DEFAULT   | ROC-AUC: 0.7217 | F1: 0.6557
LAMA_FAST      | ROC-AUC: 0.7138 | F1: 0.6552
LAMA_ADVANCED  | ROC-AUC: 0.5000 | F1: 0.0000
LAMA_FS        | ROC-AUC: 0.7217 | F1: 0.4082
--> Лучший бэйзлайн: LAMA_DEFAULT, ROC-AUC = 0.7217


Классическая конфигурация LAMA, несмотря на консерватизм, дала лучший результат среди тестируемых.

### Собственное решение 1

In [21]:
# PIPELINE 1: LightGBM + scale + auto feature selection

# PIPELINE 1A: только scaling + LGBM (без фичер селекции)
pipe_lgbm_base = Pipeline([
    ('scaler', StandardScaler()),
    ('lgbm', LGBMClassifier(n_estimators=100, random_state=42))
])
pipe_lgbm_base.fit(X_train, y_train)
pred_lgbm_base = pipe_lgbm_base.predict_proba(X_test)[:, 1]
auc_lgbm_base = roc_auc_score(y_test, pred_lgbm_base)
print(f'[Pipeline LGBM basic] ROC-AUC: {auc_lgbm_base:.4f}')

# PIPELINE 1B: scaling + feature selection + LGBM
# Селектор тренируется внутри пайплайна по cross-val на train (без data leakage)
pipe_lgbm_fs = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(LGBMClassifier(n_estimators=100, random_state=42), threshold="median")),
    ('lgbm', LGBMClassifier(n_estimators=100, random_state=42))
])
pipe_lgbm_fs.fit(X_train, y_train)
pred_lgbm_fs = pipe_lgbm_fs.predict_proba(X_test)[:, 1]
auc_lgbm_fs = roc_auc_score(y_test, pred_lgbm_fs)
print(f'[Pipeline LGBM + feature selection] ROC-AUC: {auc_lgbm_fs:.4f}')

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[Pipeline LGBM basic] ROC-AUC: 0.6980
[Pipeline LGBM + feature selection] ROC-AUC: 0.6742


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SelectFromModel was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SelectFromModel was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


### Собственное решение 2

In [22]:
# PIPELINE 2: CatBoost + простая обработка пропусков
pipe_catboost = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('catboost', CatBoostClassifier(iterations=200,
                                    learning_rate=0.05,
                                    eval_metric='AUC',
                                    random_state=42,
                                    verbose=0))
])

pipe_catboost.fit(X_train, y_train)
pred_cat = pipe_catboost.predict_proba(X_test)[:, 1]
auc_cat = roc_auc_score(y_test, pred_cat)
print(f'[Pipeline CatBoost] ROC-AUC: {auc_cat:.4f}')


[Pipeline CatBoost] ROC-AUC: 0.7048


~~### Собственное решение 3~~

In [23]:
# # PIPELINE 3: Logistic Regression + StandardScaler + гиперпараметры GridSearch
# pipe_logreg = Pipeline([
#     ('scaler', StandardScaler()),
#     ('logreg', LogisticRegression(max_iter=300))
# ])

# param_grid = {'logreg__C': [0.01, 0.1, 1, 3, 10]}
# grid = GridSearchCV(pipe_logreg, param_grid, scoring='roc_auc', cv=5)
# grid.fit(X_train, y_train)

# print(f"[Pipeline LogReg] Лучшее C: {grid.best_params_}")
# best_logreg = grid.best_estimator_
# preds_lr = best_logreg.predict_proba(X_test)[:, 1]
# auc_lr = roc_auc_score(y_test, preds_lr)
# print(f'[Pipeline LogReg tuned] ROC-AUC: {auc_lr:.4f}')

~~### Вывод~~
~~Свое решение оказалось лучше LAMA baseline: максимум ROC-AUC своего решения: 0.7455 (пайплайн 3), лучшего baseline: 0.7217~~


----
### [Дополнительно]

1) Исправление 1 - использовать не GridSearch для подбора параметров

In [24]:
!pip install optuna

In [35]:
import optuna
from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

In [31]:
import warnings
warnings.filterwarnings('ignore')

In [46]:
# Вариант 1
def objective(trial):
    try:
        pipe = Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('lgbm', LGBMClassifier(
                n_estimators=trial.suggest_int('n_estimators', 80, 300),
                learning_rate=trial.suggest_float('learning_rate', 0.01, 0.12, log=True),
                max_depth=trial.suggest_int('max_depth', 3, 8),
                num_leaves=trial.suggest_int('num_leaves', 5, 32),
                subsample=trial.suggest_float('subsample', 0.6, 1.0),
                reg_alpha=trial.suggest_float('reg_alpha', 0.0, 3.0),
                reg_lambda=trial.suggest_float('reg_lambda', 0.0, 3.0),
                random_state=42,
                n_jobs=-1,
                verbose=-1,
            ))
        ])
        auc = cross_val_score(
            pipe,
            X_train, y_train,
            scoring=make_scorer(roc_auc_score, needs_proba=True),
            cv=5
        ).mean()
        if np.isnan(auc):
            return 0.0
        return auc
    except Exception as e:
        print(e)
        return 0.0


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("Лучшие гиперпы Optuna:", study.best_trial.params)

# Финальное обучение и тест
best_params = study.best_trial.params
pipe_optuna = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('lgbm', LGBMClassifier(
        **best_params,
        random_state=42,
        n_jobs=-1,
        verbose=-1,
    )),
])
pipe_optuna.fit(X_train, y_train)
pred_optuna = pipe_optuna.predict_proba(X_test)[:, 1]
auc_optuna = roc_auc_score(y_test, pred_optuna)
print(f'[Pipeline LGBM + Optuna] ROC-AUC: {auc_optuna:.4f}')


[I 2025-12-28 22:14:51,371] A new study created in memory with name: no-name-7c149ca5-82b9-46ff-9741-6b9f30e756bb
INFO:optuna.storages._in_memory:A new study created in memory with name: no-name-7c149ca5-82b9-46ff-9741-6b9f30e756bb


  0%|          | 0/40 [00:00<?, ?it/s]

INFO:optuna.study.study:Trial 0 finished with value: 0.0 and parameters: {'n_estimators': 233, 'learning_rate': 0.04104978229918572, 'max_depth': 6, 'num_leaves': 27, 'subsample': 0.8133734315144017, 'reg_alpha': 2.257040051482158, 'reg_lambda': 2.2324026442045586}. Best is trial 0 with value: 0.0.
INFO:optuna.study.study:Trial 1 finished with value: 0.0 and parameters: {'n_estimators': 106, 'learning_rate': 0.06169932093653959, 'max_depth': 3, 'num_leaves': 24, 'subsample': 0.7795293409385702, 'reg_alpha': 2.03913365026799, 'reg_lambda': 2.2734753004261856}. Best is trial 0 with value: 0.0.


[I 2025-12-28 22:14:51,574] Trial 0 finished with value: 0.0 and parameters: {'n_estimators': 233, 'learning_rate': 0.04104978229918572, 'max_depth': 6, 'num_leaves': 27, 'subsample': 0.8133734315144017, 'reg_alpha': 2.257040051482158, 'reg_lambda': 2.2324026442045586}. Best is trial 0 with value: 0.0.
[I 2025-12-28 22:14:51,711] Trial 1 finished with value: 0.0 and parameters: {'n_estimators': 106, 'learning_rate': 0.06169932093653959, 'max_depth': 3, 'num_leaves': 24, 'subsample': 0.7795293409385702, 'reg_alpha': 2.03913365026799, 'reg_lambda': 2.2734753004261856}. Best is trial 0 with value: 0.0.


INFO:optuna.study.study:Trial 2 finished with value: 0.0 and parameters: {'n_estimators': 260, 'learning_rate': 0.08994427676806044, 'max_depth': 7, 'num_leaves': 17, 'subsample': 0.7114700765295074, 'reg_alpha': 2.5138909984763336, 'reg_lambda': 2.9293134434883115}. Best is trial 0 with value: 0.0.
INFO:optuna.study.study:Trial 3 finished with value: 0.0 and parameters: {'n_estimators': 183, 'learning_rate': 0.11385929307594019, 'max_depth': 4, 'num_leaves': 8, 'subsample': 0.9207614930617725, 'reg_alpha': 0.9179984324637993, 'reg_lambda': 2.97300514051142}. Best is trial 0 with value: 0.0.


...

[I 2025-12-28 22:14:58,147] Trial 35 finished with value: 0.0 and parameters: {'n_estimators': 251, 'learning_rate': 0.11645505159588118, 'max_depth': 5, 'num_leaves': 29, 'subsample': 0.727471117042364, 'reg_alpha': 1.0323464754448128, 'reg_lambda': 2.796971929623157}. Best is trial 0 with value: 0.0.
[I 2025-12-28 22:14:58,320] Trial 36 finished with value: 0.0 and parameters: {'n_estimators': 123, 'learning_rate': 0.05535898792383328, 'max_depth': 4, 'num_leaves': 6, 'subsample': 0.688925778608202, 'reg_alpha': 0.6098601149617151, 'reg_lambda': 1.7046531084897256}. Best is trial 0 with value: 0.0.


INFO:optuna.study.study:Trial 37 finished with value: 0.0 and parameters: {'n_estimators': 143, 'learning_rate': 0.07890427662786581, 'max_depth': 6, 'num_leaves': 20, 'subsample': 0.9480156745077905, 'reg_alpha': 2.3006448962505277, 'reg_lambda': 2.8438094361398107}. Best is trial 0 with value: 0.0.
INFO:optuna.study.study:Trial 38 finished with value: 0.0 and parameters: {'n_estimators': 97, 'learning_rate': 0.10358610143005223, 'max_depth': 3, 'num_leaves': 14, 'subsample': 0.794964815979305, 'reg_alpha': 0.4160280916826028, 'reg_lambda': 0.6227290520690993}. Best is trial 0 with value: 0.0.


[I 2025-12-28 22:14:58,498] Trial 37 finished with value: 0.0 and parameters: {'n_estimators': 143, 'learning_rate': 0.07890427662786581, 'max_depth': 6, 'num_leaves': 20, 'subsample': 0.9480156745077905, 'reg_alpha': 2.3006448962505277, 'reg_lambda': 2.8438094361398107}. Best is trial 0 with value: 0.0.
[I 2025-12-28 22:14:58,649] Trial 38 finished with value: 0.0 and parameters: {'n_estimators': 97, 'learning_rate': 0.10358610143005223, 'max_depth': 3, 'num_leaves': 14, 'subsample': 0.794964815979305, 'reg_alpha': 0.4160280916826028, 'reg_lambda': 0.6227290520690993}. Best is trial 0 with value: 0.0.


INFO:optuna.study.study:Trial 39 finished with value: 0.0 and parameters: {'n_estimators': 262, 'learning_rate': 0.0647717216495692, 'max_depth': 7, 'num_leaves': 30, 'subsample': 0.7688217738410504, 'reg_alpha': 1.4384348634334865, 'reg_lambda': 1.5609845004426663}. Best is trial 0 with value: 0.0.


[I 2025-12-28 22:14:58,877] Trial 39 finished with value: 0.0 and parameters: {'n_estimators': 262, 'learning_rate': 0.0647717216495692, 'max_depth': 7, 'num_leaves': 30, 'subsample': 0.7688217738410504, 'reg_alpha': 1.4384348634334865, 'reg_lambda': 1.5609845004426663}. Best is trial 0 with value: 0.0.
Лучшие гиперпы Optuna: {'n_estimators': 233, 'learning_rate': 0.04104978229918572, 'max_depth': 6, 'num_leaves': 27, 'subsample': 0.8133734315144017, 'reg_alpha': 2.257040051482158, 'reg_lambda': 2.2324026442045586}
[Pipeline LGBM + Optuna] ROC-AUC: 0.6776


In [52]:
# Вариант 2
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def objective_lr(trial):
    c_value = trial.suggest_float("C", 1e-1, 10.0, log=True)

    solver = trial.suggest_categorical("solver", ["liblinear", "lbfgs"])
    if solver == "liblinear":
        penalty = trial.suggest_categorical("penalty", ["l1", "l2"])
    else:
        penalty = "l2"

    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('logreg', LogisticRegression(
            C=c_value,
            solver=solver,
            penalty=penalty,
            max_iter=1000,
            random_state=42
        ))
    ])

    roc_auc = cross_val_score(pipe, X_train, y_train, cv=cv_strategy, scoring="roc_auc").mean()
    return roc_auc

optuna.logging.set_verbosity(optuna.logging.INFO)
study_lr = optuna.create_study(direction="maximize")
study_lr.optimize(objective_lr, n_trials=50, show_progress_bar=True)

print(f"Лучший ROC-AUC (CV): {study_lr.best_value:.4f}")
print("Лучшие гиперпы Optuna:", study_lr.best_params)

# Финальное обучение и тест
best_params = study_lr.best_params

final_solver = best_params['solver']
final_penalty = best_params.get('penalty', 'l2')

pipe_final_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(
        C=best_params['C'],
        solver=final_solver,
        penalty=final_penalty,
        max_iter=1000,
        random_state=42
    ))
])

pipe_final_lr.fit(X_train, y_train)
pred_final_lr = pipe_final_lr.predict_proba(X_test)[:, 1]
auc_final_lr = roc_auc_score(y_test, pred_final_lr)

print("-" * 40)
print(f'[Pipeline LogReg + Optuna] ROC-AUC: {auc_final_lr:.4f}')
print("-" * 40)

[I 2025-12-28 22:19:44,128] A new study created in memory with name: no-name-fd698e50-ef3e-461e-a635-2ff5f5b8fc88
INFO:optuna.storages._in_memory:A new study created in memory with name: no-name-fd698e50-ef3e-461e-a635-2ff5f5b8fc88


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:optuna.study.study:Trial 0 finished with value: 0.6018745143745143 and parameters: {'C': 2.2828178361498486, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.6018745143745143.
INFO:optuna.study.study:Trial 1 finished with value: 0.6229312354312354 and parameters: {'C': 0.10486052567515372, 'solver': 'liblinear', 'penalty': 'l2'}. Best is trial 1 with value: 0.6229312354312354.
INFO:optuna.study.study:Trial 2 finished with value: 0.600475912975913 and parameters: {'C': 2.6056514497797334, 'solver': 'lbfgs'}. Best is trial 1 with value: 0.6229312354312354.


[I 2025-12-28 22:19:44,234] Trial 0 finished with value: 0.6018745143745143 and parameters: {'C': 2.2828178361498486, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.6018745143745143.
[I 2025-12-28 22:19:44,301] Trial 1 finished with value: 0.6229312354312354 and parameters: {'C': 0.10486052567515372, 'solver': 'liblinear', 'penalty': 'l2'}. Best is trial 1 with value: 0.6229312354312354.
[I 2025-12-28 22:19:44,387] Trial 2 finished with value: 0.600475912975913 and parameters: {'C': 2.6056514497797334, 'solver': 'lbfgs'}. Best is trial 1 with value: 0.6229312354312354.


INFO:optuna.study.study:Trial 3 finished with value: 0.6005147630147631 and parameters: {'C': 0.4598553428370276, 'solver': 'liblinear', 'penalty': 'l2'}. Best is trial 1 with value: 0.6229312354312354.
INFO:optuna.study.study:Trial 4 finished with value: 0.6074980574980575 and parameters: {'C': 0.26836138719869695, 'solver': 'lbfgs'}. Best is trial 1 with value: 0.6229312354312354.
INFO:optuna.study.study:Trial 5 finished with value: 0.5713772338772338 and parameters: {'C': 0.3225540897159633, 'solver': 'liblinear', 'penalty': 'l1'}. Best is trial 1 with value: 0.6229312354312354.


...

[I 2025-12-28 22:19:47,363] Trial 41 finished with value: 0.6229312354312354 and parameters: {'C': 0.10555422036863131, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.622940947940948.
[I 2025-12-28 22:19:47,442] Trial 42 finished with value: 0.6215423465423465 and parameters: {'C': 0.1010188769357931, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.622940947940948.
[I 2025-12-28 22:19:47,530] Trial 43 finished with value: 0.6173465423465423 and parameters: {'C': 0.13364639135429332, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.622940947940948.


INFO:optuna.study.study:Trial 44 finished with value: 0.6159285159285159 and parameters: {'C': 0.18016125102021155, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.622940947940948.
INFO:optuna.study.study:Trial 45 finished with value: 0.6215423465423465 and parameters: {'C': 0.10288251394172904, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.622940947940948.


[I 2025-12-28 22:19:47,607] Trial 44 finished with value: 0.6159285159285159 and parameters: {'C': 0.18016125102021155, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.622940947940948.
[I 2025-12-28 22:19:47,694] Trial 45 finished with value: 0.6215423465423465 and parameters: {'C': 0.10288251394172904, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.622940947940948.
[I 2025-12-28 22:19:47,808] Trial 46 finished with value: 0.5962995337995338 and parameters: {'C': 3.8924064503994, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.622940947940948.


INFO:optuna.study.study:Trial 46 finished with value: 0.5962995337995338 and parameters: {'C': 3.8924064503994, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.622940947940948.
INFO:optuna.study.study:Trial 47 finished with value: 0.5976884226884227 and parameters: {'C': 1.091616330414342, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.622940947940948.
INFO:optuna.study.study:Trial 48 finished with value: 0.6159479409479409 and parameters: {'C': 0.15266670294821644, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.622940947940948.
INFO:optuna.study.study:Trial 49 finished with value: 0.6088966588966589 and parameters: {'C': 0.28553345541494946, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.622940947940948.


[I 2025-12-28 22:19:47,911] Trial 47 finished with value: 0.5976884226884227 and parameters: {'C': 1.091616330414342, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.622940947940948.
[I 2025-12-28 22:19:47,994] Trial 48 finished with value: 0.6159479409479409 and parameters: {'C': 0.15266670294821644, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.622940947940948.
[I 2025-12-28 22:19:48,090] Trial 49 finished with value: 0.6088966588966589 and parameters: {'C': 0.28553345541494946, 'solver': 'lbfgs'}. Best is trial 16 with value: 0.622940947940948.
Лучший ROC-AUC (CV): 0.6229
Лучшие гиперпы Optuna: {'C': 0.8190030071607937, 'solver': 'liblinear', 'penalty': 'l1'}
----------------------------------------
[Pipeline LogReg + Optuna] ROC-AUC: 0.7274
----------------------------------------


In [28]:
from sklearn.model_selection import RandomizedSearchCV

# Вариант 3 - пробуем RandomSearch
pipe = Pipeline([
    ('scaler', StandardScaler()),
    # ('pca', PCA(n_components=0.95)),
    ('lgbm', LGBMClassifier(verbose=-1, random_state=42))
])

param_dist = {
    'lgbm__n_estimators': randint(100, 500),
    'lgbm__learning_rate': uniform(0.01, 0.1),
    'lgbm__max_depth': randint(3, 7),
    'lgbm__num_leaves': randint(5, 20),
    'lgbm__subsample': uniform(0.6, 0.4),
    'lgbm__reg_alpha': uniform(0, 5),
    'lgbm__reg_lambda': uniform(0, 5)
}

rs = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_dist,
    n_iter=50,
    scoring='roc_auc',
    cv=5
    # random_state=42,
    # n_jobs=-1
)

rs.fit(X_train, y_train)
print(f"Best params: {rs.best_params_}")
best_model = rs.best_estimator_
preds_tuned = best_model.predict_proba(X_test)[:, 1]
auc_tuned = roc_auc_score(y_test, preds_tuned)
print(f'[Pipeline LGBM + RandomSearch] ROC-AUC: {auc_tuned:.4f}')

Best params: {'lgbm__learning_rate': np.float64(0.021586905952512975), 'lgbm__max_depth': 5, 'lgbm__n_estimators': 300, 'lgbm__num_leaves': 12, 'lgbm__reg_alpha': np.float64(2.247253370691017), 'lgbm__reg_lambda': np.float64(0.47705058245205656), 'lgbm__subsample': np.float64(0.7483273008793065)}
[Pipeline LGBM + RandomSearch] ROC-AUC: 0.6810


**Вывод:** `Pipeline LogReg + Optuna` дали результат, бьющий бейзлайн - 0.7274 vs 0.7217

2) Проблема 2 - пофильтерить коррелирующие фичи

In [32]:
def remove_highcorr_features(x, threshold):
    corr_matrix = x.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    print(f"Dropping {len(to_drop)} correlated features: {to_drop}")
    return x.drop(columns=to_drop), to_drop

X_train_uncorr, dropped_cols = remove_highcorr_features(X_train, 0.80)
X_test_uncorr = X_test.drop(columns=dropped_cols)

print(f"Cleaned train shape: {X_train_uncorr.shape}")

pipe_uncorr = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=300, C=1.0))
])

pipe_uncorr.fit(X_train_uncorr, y_train)
pred_uncorr = pipe_uncorr.predict_proba(X_test_uncorr)[:, 1]
auc_uncorr = roc_auc_score(y_test, pred_uncorr)
print(f'\n[Alternative solution: NO correlation] ROC-AUC: {auc_uncorr:.4f}')
print("-" * 10)


Dropping 4 correlated features: ['f4_p300', 'p4_p300', 'p8_p300', 'o2_p300']
Cleaned train shape: (120, 15)

[Alternative solution: NO correlation] ROC-AUC: 0.7387
----------


**Вывод:** в EDA писала о специфике ЭЭГ данных и что 19 каналов для этого домена - мало и для определения отклонения от нормы нужно смотреть на аномалии - вышло так, что после удаления признаков с коррелацией более 80% ROC-AUC опустился с >0.74 до 0.7387 при том же решении.

3) Проблема 3 - странности результатов я думаю связаны в первую очередь с тем, что данных очень мало, медицинские данные редки и как правило довольно шумны.

In [33]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

# лимитим max_depth и num_leaves
# добавила PCA - полезно для EEG и заодно полечит корреляционность

pipe_lgbm_constrained = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95)),
    ('lgbm', LGBMClassifier(
        n_estimators=200,
        learning_rate=0.03,   # помедленнее обучение
        num_leaves=7,         # малое количество
        max_depth=3,          # низкая граница
        reg_alpha=1,
        reg_lambda=1,
        random_state=42,
        verbose=-1
    ))
])

pipe_lgbm_constrained.fit(X_train, y_train)
pred_lgbm_fix = pipe_lgbm_constrained.predict_proba(X_test)[:, 1]
auc_lgbm_fix = roc_auc_score(y_test, pred_lgbm_fix)
print(f'[Pipeline LGBM Constrained + PCA] ROC-AUC: {auc_lgbm_fix:.4f}')

[Pipeline LGBM Constrained + PCA] ROC-AUC: 0.6867


In [34]:
# Пробуем еще воутинг:
clf1 = LogisticRegression(C=1, solver='liblinear', random_state=42)
clf2 = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
clf3 = SVC(probability=True, kernel='rbf', C=1, random_state=42)

eclf = VotingClassifier(
    estimators=[('lr', clf1), ('rf', clf2), ('svm', clf3)],
    voting='soft'
)

pipe_voting = Pipeline([
    ('scaler', StandardScaler()),
    ('voting', eclf)
])

pipe_voting.fit(X_train, y_train)
pred_vote = pipe_voting.predict_proba(X_test)[:, 1]
auc_vote = roc_auc_score(y_test, pred_vote)
print(f'[Pipeline Voting Ensemble] ROC-AUC: {auc_vote:.4f}')


[Pipeline Voting Ensemble] ROC-AUC: 0.7115
